In [1]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print(tf.__version__)
from tensorboard.plugins import projector


PATH = os.getcwd()

LOG_DIR = PATH+ '/embedding1-logs'

Instructions for updating:
non-resource variables are not supported in the long term
2.3.1


In [4]:
import pandas as pd
df = pd.read_csv("/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/Output_csv/Model_binary_v3.csv", usecols=range(1,5))
predicted_noise = df[df["Predicted_label"]==1]
cervix = df[(df["Predicted_label"] == 0)]
cervix = cervix[cervix["Data_type"] == "Train"]

test_ext = pd.read_csv("/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/TESTING_ON_5DATASETS/testing_data.csv")

imagenetdata = test_ext[test_ext["imagenet"] == 1]
blooddata = test_ext[test_ext["blood"] == 1]
eyedata = test_ext[test_ext["eye"] == 1]
skindata = test_ext[test_ext["skin"] == 1]
surgerydata = test_ext[test_ext["surgery"] == 1]

img_dirs = []
labels_temp = []
for a in list(predicted_noise["Path"]):
    img_dirs.append(a)
    labels_temp.append(0)
for b in list(cervix["Path"])[:500]:
    img_dirs.append(b)
    labels_temp.append(1)
# for j in list(imagenetdata["Path"])[:500]:
#     img_dirs.append(j)
#     labels_temp.append(2)
# for k in list(blooddata["Path"])[:500]:
#     img_dirs.append(k)
#     labels_temp.append(3)
# for l in list(eyedata["Path"])[:500]:
#     img_dirs.append(l)
#     labels_temp.append(4)
# for m in list(skindata["Path"])[:500]:
#     img_dirs.append(m)
#     labels_temp.append(5)
# for n in list(surgerydata["Path"])[:500]:
#     img_dirs.append(n)
#     labels_temp.append(6)

data1 = img_dirs
labels = np.asarray(labels_temp)
print("Length of data :",len(data1))
print("Length of labels :",len(labels))

Length of data : 1354
Length of labels : 1354


In [2]:
x = np.load("/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/UMAP/labels1.npy")

In [3]:
import pandas as pd
c = pd.read_csv("/workstation/home/bijoy/data_from_b170007ec/Programs/Bhanu/SCLEARNING/UMAP/viz.csv")
data = list(c["Path"])
labels = list(c["label"])

In [5]:
len(labels)

3403

In [6]:
#%%
from tqdm import tqdm
img_data=[]
for img in tqdm(data):
    input_img=cv2.imread(img)
    input_img_resize=cv2.resize(input_img,(128,128))
    img_data.append(input_img_resize)
    
img_data = np.array(img_data)

100%|██████████| 3403/3403 [07:03<00:00,  8.03it/s] 


In [8]:
feature_vectors = np.loadtxt('feature_vectors_samples1.txt')
feature_vectors = feature_vectors
print ("feature_vectors_shape:",feature_vectors.shape)
print ("num of images:",feature_vectors.shape[0])
print ("size of individual feature vector:",feature_vectors.shape[1])
features = tf.Variable(feature_vectors, name='features')

names = ['Cervix_Noise','cervix','imagenet','blood','eye','skin','surgery']

metadata_file = open(os.path.join(LOG_DIR, 'metadata_7_classes.tsv'), 'w')
metadata_file.write('Class\tName\n')
k=len(data)
j=0
for i in range(len(data)):
    c = names[x[i]]
    j = x[i]
    metadata_file.write('{}\t{}\n'.format(j,c))
metadata_file.close()

feature_vectors_shape: (3403, 1024)
num of images: 3403
size of individual feature vector: 1024


In [9]:
# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data
#%%
sprite = images_to_sprite(img_data)
cv2.imwrite(os.path.join(LOG_DIR, 'sprite_7_classes.png'), sprite)
#scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)

True

In [10]:
#%%
configuration = tf.ConfigProto(device_count={"GPU": 0})
with tf.Session(config=configuration) as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images_7_classes.ckpt'))
    
    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata_7_classes.tsv')
    # Comment out if you don't want sprites
    embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite_7_classes.png')
    embedding.sprite.single_image_dim.extend([img_data.shape[1], img_data.shape[1]])
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)